In [7]:
import pandas as pd
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


import categorical_embedder as ce
import numpy as np

2024-06-28 17:06:35.771188: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-28 17:06:35.772982: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-28 17:06:35.807131: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-28 17:06:35.807888: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-28 17:06:36.591342: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Num GPUs Available:  0


2024-06-28 17:06:37.517655: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
df = pd.read_csv('data/df2.csv')

In [4]:
df

,y_true,symptom,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,...,DE,De,dE,de,SERIOUS Y,SERIOUS N,manufacturer_JANSSEN,manufacturer_MODERNA,manufacturer_NOVAVAX,manufacturer_PFIZER\BIONTECH
0,1,Angioedema,23,95,161,304,308,287,191,105,...,2878.0,943022.0,2351.0,1530710.0,564.0,2314.0,0,0,0,1
1,1,Hypoaesthesia,39,732,2295,4248,4426,3566,1960,862,...,26529.0,919371.0,27845.0,1505216.0,7405.0,19126.0,0,0,0,1
2,1,Erythema multiforme,20,12,31,28,41,30,34,30,...,733.0,945167.0,2228.0,1530833.0,131.0,602.0,0,0,0,1
3,1,Insomnia,33,205,732,1345,1345,1253,1012,581,...,11215.0,934685.0,14609.0,1518452.0,2674.0,8541.0,0,0,0,1
4,1,Myalgia,106,935,4266,7133,6992,6599,4653,2335,...,52182.0,893718.0,75910.0,1457151.0,12832.0,39351.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0,Increased insulin requirement,0,0,1,1,0,0,1,0,...,3.0,98389.0,33.0,2380536.0,2.0,1.0,1,0,0,0
1121,0,Blood lactate dehydrogenase decreased,0,0,0,0,1,0,0,0,...,7.0,945893.0,17.0,1533044.0,4.0,3.0,0,0,0,1
1122,0,Breast cellulitis,0,0,0,0,2,0,0,2,...,4.0,537275.0,16.0,1941666.0,1.0,3.0,0,1,0,0
1123,0,Vascular graft thrombosis,0,0,0,0,0,2,0,1,...,3.0,537276.0,10.0,1941672.0,3.0,0.0,0,1,0,0


In [5]:
X = df[['symptom']]
y = df['y_true']
print(X.shape)
print(y.shape)

(1125, 1)
(1125,)


In [6]:
y.values

array([1, 1, 1, ..., 0, 0, 0])

In [7]:
embedding_info = ce.get_embedding_info(X)
X_encoded, encoders = ce.get_label_encoded_data(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y)

# Get embeddings
embeddings = ce.get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info,
                               is_classification=True, epochs=100, batch_size=256)


Epoch 1/100
3/3 [==============================] - 1s 81ms/step - loss: 0.6735 - accuracy: 0.5861 - val_loss: 0.6006 - val_accuracy: 0.9112
Epoch 2/100
3/3 [==============================] - 0s 20ms/step - loss: 0.5792 - accuracy: 0.8798 - val_loss: 0.4915 - val_accuracy: 0.9112
Epoch 3/100
3/3 [==============================] - 0s 20ms/step - loss: 0.4706 - accuracy: 0.8798 - val_loss: 0.3789 - val_accuracy: 0.9112
Epoch 4/100
3/3 [==============================] - 0s 31ms/step - loss: 0.3707 - accuracy: 0.8798 - val_loss: 0.3056 - val_accuracy: 0.9112
Epoch 5/100
3/3 [==============================] - 0s 26ms/step - loss: 0.3233 - accuracy: 0.8798 - val_loss: 0.2944 - val_accuracy: 0.9112
Epoch 6/100
3/3 [==============================] - 0s 29ms/step - loss: 0.3207 - accuracy: 0.8798 - val_loss: 0.2986 - val_accuracy: 0.9112
Epoch 7/100
3/3 [==============================] - 0s 29ms/step - loss: 0.2920 - accuracy: 0.8798 - val_loss: 0.2900 - val_accuracy: 0.9112
Epoch 8/100
3/3 [===

In [8]:
df_embedded = ce.fit_transform(df, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
df_embedded.head()

,y_true,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,...,symptom_embedding_40,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49
0,1,23,95,161,304,308,287,191,105,45,...,0.074551,-0.038803,0.031955,-0.013366,0.006447,0.082953,0.059780,-0.082158,-0.033914,0.011456
1,1,39,732,2295,4248,4426,3566,1960,862,241,...,0.033519,-0.002073,0.025551,-0.050798,0.006093,0.075400,0.073157,-0.099012,-0.017383,0.000349
2,1,20,12,31,28,41,30,34,30,20,...,0.011984,-0.023250,-0.000355,-0.020413,0.035789,-0.032599,-0.004464,0.015447,0.040009,0.041251
3,1,33,205,732,1345,1345,1253,1012,581,180,...,0.004370,0.004094,0.027650,-0.049678,-0.039587,0.058385,0.030067,-0.059521,-0.070503,-0.013063
4,1,106,935,4266,7133,6992,6599,4653,2335,583,...,0.042239,-0.031789,0.028889,-0.047447,-0.007253,0.101274,0.050122,-0.026549,-0.051247,-0.007401


In [9]:
num_attribs = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99', '100-109',
               '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3', 'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
               'NUMDAYS_12-13', 'NUMDAYS_14+', 'DIED', 'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT',
               'DE', 'De', 'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'SEX_M', 'SEX_F', 'SEX_U']

# Separate features and target
X = df_embedded.drop(columns=['y_true'])
y_true = df_embedded['y_true']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('min_max_scaler', MinMaxScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs)
], remainder='passthrough')

# Apply transformation
X_transformed = full_pipeline.fit_transform(X)

# Combine the transformed features with the target
df_transformed = pd.DataFrame(X_transformed, columns=X.columns)

In [10]:
df_transformed.columns

Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90-99', '100-109', '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3',
       'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
       'NUMDAYS_12-13', 'NUMDAYS_14+', 'SEX_M', 'SEX_F', 'SEX_U', 'DIED',
       'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT', 'DE', 'De',
       'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'manufacturer_JANSSEN',
       'manufacturer_MODERNA', 'manufacturer_NOVAVAX',
       'manufacturer_PFIZER\BIONTECH', 'symptom_embedding_0',
       'symptom_embedding_1', 'symptom_embedding_2', 'symptom_embedding_3',
       'symptom_embedding_4', 'symptom_embedding_5', 'symptom_embedding_6',
       'symptom_embedding_7', 'symptom_embedding_8', 'symptom_embedding_9',
       'symptom_embedding_10', 'symptom_embedding_11', 'symptom_embedding_12',
       'symptom_embedding_13', 'symptom_embedding_14', 'symptom_embedding_15',
       'symptom_embedding_16', 'symptom_embedding_1

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df_transformed, y_true, test_size=0.2, random_state=42)

X_train.to_csv('../Machine_Learning/X_train.csv', index=False)
X_test.to_csv('../Machine_Learning/X_test.csv', index=False)
y_train.to_csv('../Machine_Learning/y_train.csv', index=False)
y_test.to_csv('../Machine_Learning/y_test.csv', index=False)

# Generate splits with Moderna = test

In [11]:
df_transformed['y_true'] = y_true

In [12]:
df_transformed

,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99,...,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49,y_true
0,0.020193,0.016728,0.014843,0.017429,0.017247,0.016329,0.012564,0.012005,0.013682,0.006903,...,-0.038803,0.031955,-0.013366,0.006447,0.082953,0.059780,-0.082158,-0.033914,0.011456,1
1,0.034241,0.128896,0.211579,0.243550,0.247844,0.202890,0.128930,0.098559,0.073275,0.034517,...,-0.002073,0.025551,-0.050798,0.006093,0.075400,0.073157,-0.099012,-0.017383,0.000349,1
2,0.017559,0.002113,0.002858,0.001605,0.002296,0.001707,0.002237,0.003430,0.006081,0.003945,...,-0.023250,-0.000355,-0.020413,0.035789,-0.032599,-0.004464,0.015447,0.040009,0.041251,1
3,0.028973,0.036098,0.067484,0.077113,0.075316,0.071290,0.066570,0.066430,0.054728,0.023669,...,0.004094,0.027650,-0.049678,-0.039587,0.058385,0.030067,-0.059521,-0.070503,-0.013063,1
4,0.093064,0.164642,0.393288,0.408955,0.391533,0.375455,0.306078,0.266979,0.177258,0.074951,...,-0.031789,0.028889,-0.047447,-0.007253,0.101274,0.050122,-0.026549,-0.051247,-0.007401,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0.000000,0.000000,0.000092,0.000057,0.000000,0.000000,0.000066,0.000000,0.000000,0.000000,...,0.032908,-0.051052,-0.013448,-0.031592,0.058226,-0.018147,0.039280,0.011030,-0.027882,0
1121,0.000000,0.000000,0.000000,0.000000,0.000056,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.040171,-0.017807,-0.028698,-0.025172,-0.025134,-0.015200,-0.036714,-0.047624,0.026188,0
1122,0.000000,0.000000,0.000000,0.000000,0.000112,0.000000,0.000000,0.000229,0.000000,0.000000,...,0.032199,-0.003319,0.031135,0.060899,-0.070487,-0.024701,-0.056912,0.026682,-0.029919,0
1123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000114,0.000000,0.000114,0.000000,0.000000,...,0.038854,0.024804,0.028361,0.009756,-0.046723,-0.036647,-0.060717,0.015059,0.039474,0


In [13]:
X_test = df_transformed[(df_transformed['manufacturer_MODERNA'] == 1)]
y_test = X_test['y_true']
X_test = X_test.drop(columns=['y_true'])

X_train = df_transformed[(df_transformed['manufacturer_MODERNA'] == 0)]
y_train = X_train['y_true']
X_train = X_train.drop(columns=['y_true'])

In [14]:
X_train.to_csv('../Machine_Learning/X_train_Moderna.csv', index=False)
X_test.to_csv('../Machine_Learning/X_test_Moderna.csv', index=False)
y_train.to_csv('../Machine_Learning/y_train_Moderna.csv', index=False)
y_test.to_csv('../Machine_Learning/y_test_Moderna.csv', index=False)

# With Janssen as Test

In [15]:
X_test = df_transformed[(df_transformed['manufacturer_JANSSEN'] == 1)]
y_test = X_test['y_true']
X_test = X_test.drop(columns=['y_true'])

X_train = df_transformed[(df_transformed['manufacturer_JANSSEN'] == 0)]
y_train = X_train['y_true']
X_train = X_train.drop(columns=['y_true'])

In [16]:
X_train.to_csv('../Machine_Learning/X_train_Janssen.csv', index=False)
X_test.to_csv('../Machine_Learning/X_test_Janssen.csv', index=False)
y_train.to_csv('../Machine_Learning/y_train_Janssen.csv', index=False)
y_test.to_csv('../Machine_Learning/y_test_Janssen.csv', index=False)

# Create Embeddings for Dataset where a symptom only appears in test or train

### fist we want to know if there are any unique adverse reaction across the vainnes to split the dataset accoringly

In [1]:
Pfizer_adverse_reactions = [
    "Vomiting",
    "Lymphadenopathy",
    "Rash", 
    "Pruritus", 
    "Urticaria", 
    "Angioedema",
    "Anaphylaxis",
    "Decreased appetite",
    "Insomnia",
    "Headache",
    "Dizziness",
    "Lethargy",
    "Facial paralysis", 
    "Paraesthesia",
    "Hypoaesthesia",
    "Myocarditis",
    "Pericarditis",
    "Diarrhoea",
    "Nausea",
    "Hyperhidrosis",
    "Night sweats",
    "Erythema multiforme",
    "Arthralgia",
    "Myalgia",
    "Pain in extremity",
    "Heavy menstrual bleeding",
    "Injection site pain",
    "Fatigue",
    "Chills",
    "Pyrexia",
    "Injection site swelling",
    "Injection site redness",
    "Asthenia",
    "Malaise",
    "Injection site pruritus",
    "Extensive swelling of vaccinated limb",
    "Facial swelling"
]
Moderna_adverse_reactions = [
    "Lymphadenopathy",
    "Anaphylaxis",
    "Hypersensitivity",
    "Decreased appetite",
    "Irritability", 
    "Crying",
    "Headache",
    "Sleepiness",
    "Dizziness",
    "Facial paralysis", 
    "Hypoaesthesia",
    "Paraesthesia",
    "Myocarditis",
    "Pericarditis",
    "Nausea", 
    "Vomiting",
    "Diarrhoea",
    "Abdominal pain",
    "Rash",
    "Urticaria",
    "Erythema multiforme",
    "Mechanical urticaria",
    "Chronic urticaria",
    "Myalgia",
    "Arthralgia",
    "Heavy menstrual bleeding",
    "Injection site pain",
    "Fatigue",
    "Chills",
    "Pyrexia",
    "Injection site swelling",
    "Injection site erythema",
    "Injection site urticaria",
    "Injection site rash",
    "Delayed injection site reaction",
    "Injection site pruritus",
    "Facial swelling",
    "Extensive swelling of vaccinated limb"
]

Janssen_adverse_reactions = [
    "Lymphadenopathy",
    "Immune thrombocytopenia",
    "Anaphylaxis",
    "Headache",
    "Dizziness",
    "Tremor",
    "Urticaria",
    "Hypersensitivity",
    "Paraesthesia",
    "Hypoaesthesia",
    "Facial paralysis",
    "Tinnitus",
    "Guillain-Barre syndrome",
    "Myelitis", 
    "Venous thromboembolism",
    "Thrombosis in combination with thrombocytopenia",
    "Myocarditis", 
    "Pericarditis",
    "Capillary leak syndrome",
    "Cutaneous vasculitis", 
    "Nausea",
    "Cough", 
    "Oropharyngeal pain",
    "Sneezing",
    "Diarrhoea",
    "Vomiting",
    "Rash",
    "Hyperhidrosis",
    "Myalgia",
    "Arthralgia",
    "Muscular weakness",
    "Back pain",
    "Pain in extremity",
    "Fatigue",
    "Injection site pain",
    "Injection site swelling",
    "Chills",
    "Pyrexia"
]

Novavax_adverse_reactions = [
    "Headache",
    "Nausea",
    "Vomiting",
    "Myalgia",
    "Arthralgia",
    "Injection site tenderness",
    "Injection site pain",
    "Fatigue",
    "Malaise",
    "Injection site redness",
    "Injection site swelling",
    "Pyrexia",
    "Pain in extremity",
    "Injection site pruritus",
    "Chills",
    "Injection site warmth",
    "Lymphadenopathy",
    "Anaphylaxis",
    "Paraesthesia",
    "Hypoaesthesia",
    "Myocarditis",
    "Pericarditis",
    "Hypertension",
    "Rash",
    "Erythema",
    "Pruritus",
    "Urticaria"
]

In [2]:
from collections import Counter, defaultdict

named_lists = [
    ("Pfizer", Pfizer_adverse_reactions),
    ("Moderna", Moderna_adverse_reactions),
    ("Janssen", Janssen_adverse_reactions),
    ("Novavax", Novavax_adverse_reactions)
]

# Combine all lists into one
combined_list = []
for name, lst in named_lists:
    combined_list.extend(lst)

# Count the frequency of each word
word_counts = Counter(combined_list)

# Track the lists each word appears in
word_occurrences = defaultdict(set)

for name, lst in named_lists:
    for word in lst:
        word_occurrences[word].add(name)

# Print the word counts and the lists they appear in
for word, count in word_counts.items():
    lists_containing_word = ", ".join(word_occurrences[word])
    print(f"{word}: {count} times, appears in {lists_containing_word}")

Vomiting: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Lymphadenopathy: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Rash: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Pruritus: 2 times, appears in Pfizer, Novavax
Urticaria: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Angioedema: 1 times, appears in Pfizer
Anaphylaxis: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Decreased appetite: 2 times, appears in Moderna, Pfizer
Insomnia: 1 times, appears in Pfizer
Headache: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Dizziness: 3 times, appears in Moderna, Pfizer, Janssen
Lethargy: 1 times, appears in Pfizer
Facial paralysis: 3 times, appears in Moderna, Pfizer, Janssen
Paraesthesia: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Hypoaesthesia: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Myocarditis: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Pericarditis: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Dia

In [3]:
# Combine all lists into one
combined_list = []
for name, lst in named_lists:
    combined_list.extend(lst)

# Count the frequency of each word
word_counts = Counter(combined_list)

# Track the lists each word appears in
word_occurrences = defaultdict(set)

for name, lst in named_lists:
    for word in lst:
        word_occurrences[word].add(name)

# Print the word counts and the lists they appear in
print("Word counts and lists they appear in:")
for word, count in word_counts.items():
    lists_containing_word = ", ".join(word_occurrences[word])
    print(f"{word}: {count} times, appears in {lists_containing_word}")

# Function to print words that appear in a specific number of lists
def print_words_appearing_in_n_lists(n, lst_name, lst):
    words_in_n_lists = [word for word in lst if len(word_occurrences[word]) == n]
    words_and_lists = {word: word_occurrences[word] for word in words_in_n_lists}
    print(f"Words that appear in exactly {n} list(s): {len(words_in_n_lists)}")
    for word, lists in words_and_lists.items():
        print(f"  - {word}: appears in {', '.join(lists)}")

# Show per list how many words appear in exactly one, two, three, or four lists and which lists they appear in
for name, lst in named_lists:
    print(f"\n{name}:")
    print_words_appearing_in_n_lists(1, name, lst)
    print_words_appearing_in_n_lists(2, name, lst)
    print_words_appearing_in_n_lists(3, name, lst)
    print_words_appearing_in_n_lists(4, name, lst)

Word counts and lists they appear in:
Vomiting: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Lymphadenopathy: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Rash: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Pruritus: 2 times, appears in Pfizer, Novavax
Urticaria: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Angioedema: 1 times, appears in Pfizer
Anaphylaxis: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Decreased appetite: 2 times, appears in Moderna, Pfizer
Insomnia: 1 times, appears in Pfizer
Headache: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Dizziness: 3 times, appears in Moderna, Pfizer, Janssen
Lethargy: 1 times, appears in Pfizer
Facial paralysis: 3 times, appears in Moderna, Pfizer, Janssen
Paraesthesia: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Hypoaesthesia: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Myocarditis: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Pericarditis: 4 times, appears in

In [4]:
len(combined_list)

140

In [5]:
# Combine all lists into one
combined_list = []
for name, lst in named_lists:
    combined_list.extend(lst)

# Count the frequency of each word
word_counts = Counter(combined_list)

# Track the lists each word appears in
word_occurrences = defaultdict(set)

for name, lst in named_lists:
    for word in lst:
        word_occurrences[word].add(name)

# Print the word counts and the lists they appear in
print("Word counts and lists they appear in:")
for word, count in word_counts.items():
    lists_containing_word = ", ".join(word_occurrences[word])
    print(f"{word}: {count} times, appears in {lists_containing_word}")

# Show the total number of symptoms that appear one, two, three, and four times
appear_once = sum(1 for word in word_occurrences if len(word_occurrences[word]) == 1)
appear_twice = sum(1 for word in word_occurrences if len(word_occurrences[word]) == 2)
appear_thrice = sum(1 for word in word_occurrences if len(word_occurrences[word]) == 3)
appear_four_times = sum(1 for word in word_occurrences if len(word_occurrences[word]) == 4)

print("\nTotal number of symptoms appearing in lists:")
print(f"Appear in exactly one list: {appear_once}")
print(f"Appear in exactly two lists: {appear_twice}")
print(f"Appear in exactly three lists: {appear_thrice}")
print(f"Appear in exactly four lists: {appear_four_times}")

Word counts and lists they appear in:
Vomiting: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Lymphadenopathy: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Rash: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Pruritus: 2 times, appears in Pfizer, Novavax
Urticaria: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Angioedema: 1 times, appears in Pfizer
Anaphylaxis: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Decreased appetite: 2 times, appears in Moderna, Pfizer
Insomnia: 1 times, appears in Pfizer
Headache: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Dizziness: 3 times, appears in Moderna, Pfizer, Janssen
Lethargy: 1 times, appears in Pfizer
Facial paralysis: 3 times, appears in Moderna, Pfizer, Janssen
Paraesthesia: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Hypoaesthesia: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Myocarditis: 4 times, appears in Moderna, Pfizer, Novavax, Janssen
Pericarditis: 4 times, appears in

In [26]:
df = pd.read_csv('data/df2_unique_5.csv')

In [27]:
df_test = df[df['manufacturer_JANSSEN'] == 1]

In [28]:
df_test

,y_true,symptom,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,...,DE,De,dE,de,SERIOUS Y,SERIOUS N,manufacturer_JANSSEN,manufacturer_MODERNA,manufacturer_NOVAVAX,manufacturer_PFIZER\BIONTECH
0,1,Back pain,1,56,283,367,384,383,249,80,...,2125.0,96267.0,32151.0,2348418.0,375.0,1750.0,1,0,0,0
1,1,Nausea,7,371,1817,1827,1501,1533,921,263,...,9791.0,88601.0,156847.0,2223722.0,1272.0,8519.0,1,0,0,0
2,1,Muscular weakness,0,23,95,196,198,245,168,78,...,1198.0,97194.0,24180.0,2356389.0,345.0,853.0,1,0,0,0
3,1,Facial paralysis,0,3,26,55,67,83,42,30,...,445.0,97947.0,10246.0,2370323.0,223.0,222.0,1,0,0,0
4,1,Cutaneous vasculitis,0,0,0,4,0,1,4,2,...,32.0,98360.0,461.0,2380108.0,13.0,19.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,0,Compression garment application,0,0,0,0,6,3,0,0,...,9.0,98383.0,52.0,2380517.0,8.0,1.0,1,0,0,0
556,0,Injection site urticaria,0,3,4,5,4,7,2,0,...,27.0,98365.0,5814.0,2374755.0,1.0,26.0,1,0,0,0
557,0,Tuberculin test,0,0,1,2,0,1,0,0,...,6.0,98386.0,177.0,2380392.0,1.0,5.0,1,0,0,0
558,0,Jaundice neonatal,0,0,0,0,0,0,0,0,...,4.0,98388.0,75.0,2380494.0,4.0,0.0,1,0,0,0


In [44]:
df_other = df[df['manufacturer_JANSSEN'] != 1]

In [45]:
df_other

,y_true,symptom,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,...,DE,De,dE,de,SERIOUS Y,SERIOUS N,manufacturer_JANSSEN,manufacturer_MODERNA,manufacturer_NOVAVAX,manufacturer_PFIZER\BIONTECH
560,1,Angioedema,23,95,161,304,308,287,191,105,...,2878.0,943022.0,2351.0,1530710.0,564.0,2314.0,0,0,0,1
561,1,Erythema multiforme,20,12,31,28,41,30,34,30,...,733.0,945167.0,2228.0,1530833.0,131.0,602.0,0,0,0,1
562,1,Insomnia,33,205,732,1345,1345,1253,1012,581,...,11215.0,934685.0,14609.0,1518452.0,2674.0,8541.0,0,0,0,1
563,1,Vomiting,833,2633,3123,3720,3366,3417,2721,1689,...,31787.0,914113.0,62395.0,1470666.0,9110.0,22677.0,0,0,0,1
564,1,Malaise,166,1244,2818,4085,4136,4262,3578,2473,...,40198.0,905702.0,44938.0,1488123.0,13119.0,27083.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,0,Calculus urinary,0,2,2,1,2,3,6,1,...,23.0,945877.0,18.0,1533043.0,15.0,8.0,0,0,0,1
1476,0,CSF virus no organisms observed,0,0,1,0,2,0,1,0,...,4.0,945896.0,81.0,1532980.0,4.0,0.0,0,0,0,1
1477,0,Emergency care,0,0,3,3,2,2,0,0,...,33.0,945867.0,28.0,1533033.0,23.0,10.0,0,0,0,1
1478,0,Periventricular leukomalacia,0,0,0,0,0,0,0,1,...,7.0,945893.0,5.0,1533056.0,5.0,2.0,0,0,0,1


In [29]:
symptoms = df_other["symptom"]

In [30]:
symptoms

0                             Back pain
1                                Nausea
2                     Muscular weakness
3                      Facial paralysis
4                  Cutaneous vasculitis
                     ...               
1475                   Calculus urinary
1476    CSF virus no organisms observed
1477                     Emergency care
1478       Periventricular leukomalacia
1479            Aplasia cutis congenita
Name: symptom, Length: 1480, dtype: object

In [31]:
X = df[['symptom']]
y = df['y_true']
print(X.shape)
print(y.shape)

(1480, 1)
(1480,)


In [32]:
y.values

array([1, 1, 1, ..., 0, 0, 0])

In [33]:
embedding_info = ce.get_embedding_info(X)
X_encoded, encoders = ce.get_label_encoded_data(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y)

# Get embeddings
embeddings = ce.get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info,
                               is_classification=True, epochs=100, batch_size=256)


Epoch 1/100
4/4 [==============================] - 1s 46ms/step - loss: 0.6474 - accuracy: 0.7421 - val_loss: 0.5419 - val_accuracy: 0.9459
Epoch 2/100
4/4 [==============================] - 0s 21ms/step - loss: 0.4776 - accuracy: 0.9527 - val_loss: 0.3625 - val_accuracy: 0.9459
Epoch 3/100
4/4 [==============================] - 0s 19ms/step - loss: 0.2901 - accuracy: 0.9527 - val_loss: 0.2286 - val_accuracy: 0.9459
Epoch 4/100
4/4 [==============================] - 0s 17ms/step - loss: 0.1816 - accuracy: 0.9527 - val_loss: 0.2121 - val_accuracy: 0.9459
Epoch 5/100
4/4 [==============================] - 0s 18ms/step - loss: 0.1724 - accuracy: 0.9527 - val_loss: 0.2361 - val_accuracy: 0.9459
Epoch 6/100
4/4 [==============================] - 0s 18ms/step - loss: 0.1778 - accuracy: 0.9527 - val_loss: 0.2395 - val_accuracy: 0.9459
Epoch 7/100
4/4 [==============================] - 0s 19ms/step - loss: 0.1531 - accuracy: 0.9527 - val_loss: 0.2223 - val_accuracy: 0.9459
Epoch 8/100
4/4 [===

In [34]:
df_embedded = ce.fit_transform(df, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
df_embedded.head()

,y_true,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,...,symptom_embedding_40,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49
0,1,1,56,283,367,384,383,249,80,17,...,0.010842,0.011697,0.019243,-0.001846,0.011925,-0.012115,0.041406,-0.005497,0.033912,0.044342
1,1,7,371,1817,1827,1501,1533,921,263,68,...,-0.010592,-0.045415,0.017299,-0.036043,0.007715,0.022034,0.027326,0.039226,0.011425,0.028820
2,1,0,23,95,196,198,245,168,78,22,...,0.031113,-0.019374,0.008470,0.047803,-0.036335,0.033914,0.031072,0.027121,-0.020115,0.042901
3,1,0,3,26,55,67,83,42,30,12,...,-0.046572,0.027583,-0.002554,-0.030721,0.002204,0.041874,0.020572,0.005744,-0.038501,-0.045431
4,1,0,0,0,4,0,1,4,2,1,...,-0.008580,-0.028080,-0.004603,-0.009645,-0.032794,0.009013,0.015933,0.037741,0.005871,-0.031514


In [35]:
num_attribs = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99', '100-109',
               '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3', 'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
               'NUMDAYS_12-13', 'NUMDAYS_14+', 'DIED', 'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT',
               'DE', 'De', 'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'SEX_M', 'SEX_F', 'SEX_U']

# Separate features and target
X = df_embedded.drop(columns=['y_true'])
y_true = df_embedded['y_true']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('min_max_scaler', MinMaxScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs)
], remainder='passthrough')

# Apply transformation
X_transformed = full_pipeline.fit_transform(X)

# Combine the transformed features with the target
df_transformed = pd.DataFrame(X_transformed, columns=X.columns)

In [36]:
df_transformed.columns

Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90-99', '100-109', '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3',
       'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
       'NUMDAYS_12-13', 'NUMDAYS_14+', 'SEX_M', 'SEX_F', 'SEX_U', 'DIED',
       'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT', 'DE', 'De',
       'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'manufacturer_JANSSEN',
       'manufacturer_MODERNA', 'manufacturer_NOVAVAX',
       'manufacturer_PFIZER\BIONTECH', 'symptom_embedding_0',
       'symptom_embedding_1', 'symptom_embedding_2', 'symptom_embedding_3',
       'symptom_embedding_4', 'symptom_embedding_5', 'symptom_embedding_6',
       'symptom_embedding_7', 'symptom_embedding_8', 'symptom_embedding_9',
       'symptom_embedding_10', 'symptom_embedding_11', 'symptom_embedding_12',
       'symptom_embedding_13', 'symptom_embedding_14', 'symptom_embedding_15',
       'symptom_embedding_16', 'symptom_embedding_1

In [37]:
df_transformed

,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99,...,symptom_embedding_40,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49
0,0.001200,0.021269,0.090618,0.069429,0.071602,0.078774,0.042470,0.016992,0.008947,0.006885,...,0.010842,0.011697,0.019243,-0.001846,0.011925,-0.012115,0.041406,-0.005497,0.033912,0.044342
1,0.008403,0.140904,0.581812,0.345630,0.279881,0.315302,0.157087,0.055862,0.035789,0.030981,...,-0.010592,-0.045415,0.017299,-0.036043,0.007715,0.022034,0.027326,0.039226,0.011425,0.028820
2,0.000000,0.008735,0.030419,0.037079,0.036920,0.050391,0.028654,0.016568,0.011579,0.005164,...,0.031113,-0.019374,0.008470,0.047803,-0.036335,0.033914,0.031072,0.027121,-0.020115,0.042901
3,0.000000,0.001139,0.008325,0.010405,0.012493,0.017071,0.007164,0.006372,0.006316,0.003442,...,-0.046572,0.027583,-0.002554,-0.030721,0.002204,0.041874,0.020572,0.005744,-0.038501,-0.045431
4,0.000000,0.000000,0.000000,0.000757,0.000000,0.000206,0.000682,0.000425,0.000526,0.000000,...,-0.008580,-0.028080,-0.004603,-0.009645,-0.032794,0.009013,0.015933,0.037741,0.005871,-0.031514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,0.000000,0.000760,0.000640,0.000189,0.000373,0.000617,0.001023,0.000212,0.000000,0.000000,...,0.007830,-0.030356,0.004342,-0.028587,-0.043536,-0.038061,0.012447,0.026736,-0.010893,0.012501
1476,0.000000,0.000000,0.000320,0.000000,0.000373,0.000000,0.000171,0.000000,0.000000,0.000000,...,0.048389,0.061328,0.013207,0.069063,0.051846,0.056766,0.026998,-0.078513,-0.071628,-0.061935
1477,0.000000,0.000000,0.000961,0.000568,0.000373,0.000411,0.000000,0.000000,0.000000,0.000000,...,-0.049023,0.004299,-0.067632,-0.008794,-0.006129,0.032321,-0.012683,-0.039179,-0.065106,0.021028
1478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000212,0.000000,0.000000,...,0.038412,0.018404,-0.075071,0.051485,0.034077,-0.047138,-0.025771,0.028097,0.007354,0.008311


In [38]:
df_transformed['y_true'] = y_true

In [39]:
df_transformed

,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99,...,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49,y_true
0,0.001200,0.021269,0.090618,0.069429,0.071602,0.078774,0.042470,0.016992,0.008947,0.006885,...,0.011697,0.019243,-0.001846,0.011925,-0.012115,0.041406,-0.005497,0.033912,0.044342,1
1,0.008403,0.140904,0.581812,0.345630,0.279881,0.315302,0.157087,0.055862,0.035789,0.030981,...,-0.045415,0.017299,-0.036043,0.007715,0.022034,0.027326,0.039226,0.011425,0.028820,1
2,0.000000,0.008735,0.030419,0.037079,0.036920,0.050391,0.028654,0.016568,0.011579,0.005164,...,-0.019374,0.008470,0.047803,-0.036335,0.033914,0.031072,0.027121,-0.020115,0.042901,1
3,0.000000,0.001139,0.008325,0.010405,0.012493,0.017071,0.007164,0.006372,0.006316,0.003442,...,0.027583,-0.002554,-0.030721,0.002204,0.041874,0.020572,0.005744,-0.038501,-0.045431,1
4,0.000000,0.000000,0.000000,0.000757,0.000000,0.000206,0.000682,0.000425,0.000526,0.000000,...,-0.028080,-0.004603,-0.009645,-0.032794,0.009013,0.015933,0.037741,0.005871,-0.031514,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,0.000000,0.000760,0.000640,0.000189,0.000373,0.000617,0.001023,0.000212,0.000000,0.000000,...,-0.030356,0.004342,-0.028587,-0.043536,-0.038061,0.012447,0.026736,-0.010893,0.012501,0
1476,0.000000,0.000000,0.000320,0.000000,0.000373,0.000000,0.000171,0.000000,0.000000,0.000000,...,0.061328,0.013207,0.069063,0.051846,0.056766,0.026998,-0.078513,-0.071628,-0.061935,0
1477,0.000000,0.000000,0.000961,0.000568,0.000373,0.000411,0.000000,0.000000,0.000000,0.000000,...,0.004299,-0.067632,-0.008794,-0.006129,0.032321,-0.012683,-0.039179,-0.065106,0.021028,0
1478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000212,0.000000,0.000000,...,0.018404,-0.075071,0.051485,0.034077,-0.047138,-0.025771,0.028097,0.007354,0.008311,0


In [40]:
Janssen = df_transformed[df_transformed['manufacturer_JANSSEN'] == 1]
Others  = df_transformed[df_transformed['manufacturer_JANSSEN'] == 0]

In [41]:
target_column = 'y_true'

# Split the OTHERS_df into X_train and y_train
X_train = Others.drop(columns=[target_column])
y_train = Others[target_column]

# Split the Janssen_df into X_test and y_test
X_test = Janssen.drop(columns=[target_column])
y_test = Janssen[target_column]

In [42]:
y_train

560     1
561     1
562     1
563     1
564     1
       ..
1475    0
1476    0
1477    0
1478    0
1479    0
Name: y_true, Length: 920, dtype: int64

In [43]:
X_train.to_csv('../Machine_Learning/X_train_unique_5.csv', index=False)
X_test.to_csv('../Machine_Learning/X_test_unique_5.csv', index=False)
y_train.to_csv('../Machine_Learning/y_train_unique_5.csv', index=False)
y_test.to_csv('../Machine_Learning/y_test_unique_5.csv', index=False)